In [8]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [9]:
year=2020

In [10]:
from geographiclib.geodesic import Geodesic
import pandas as pd
from lxml import etree as ET
import os

In [11]:
###set working directory
os.chdir('insert directory here')

In [12]:
#set up kml structure:
kml = ET.Element('kml')
document = ET.SubElement(kml, 'Document')

#read data
data = pd.read_csv(os.getcwd() + '/data/footprint_cost/ICLR/footprint_calcs_output/attendee_footprint_cost_ICLR.csv')
data=data.loc[data['Year']==year]
data=data.loc[data['Number']!=0]

In [14]:
for x in (data.index):
    olat = data.loc[x,'Origin_Latitude']
    olon = data.loc[x,'Origin_Longitude']
    dlat = data.loc[x,'Destination_Latitude']
    dlon = data.loc[x,'Destination_Longitude']
    route = data.loc[x,'Work Location']
       
    p=Geodesic.WGS84.Inverse(olat, olon, dlat, dlon)
    l=Geodesic.WGS84.Line(p['lat1'],p['lon1'],p['azi1'])
    if (p['s12'] >= 1000000):
        num = int (p['s12']/100000)  #number of waypoints depending on length
    else:
        num=10
    output=''
    for i in range(num+1):
        b=l.Position(i*p['s12']/num, Geodesic.STANDARD | Geodesic.LONG_UNROLL)
        output += repr(b['lon2']) + "," + repr(b['lat2']) + ",0 "
    
    
    # inner part of kml file:
    placemark = ET.SubElement(document, 'Placemark')
   
    name = ET.SubElement(placemark, 'name')
    name.text = route
    description = ET.SubElement(placemark, 'description')
    description.text = 'route ID: '+str(x)+'; distance: '+str(p['s12'])
    linestring = ET.SubElement(placemark, 'LineString')
    coordinates = ET.SubElement(linestring, 'coordinates')
    coordinates.text = output

    #complete kml file and save:
    tree = ET.ElementTree(kml)
    tree.write(os.getcwd() + '/figures/Fig 1/data/flight_paths/greatcircleflights_ICLR_' + str(year) + '.kml', pretty_print=True, xml_declaration=True,   encoding="utf-8")